# 1. 데이터 살펴보자

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 경로: /content/drive/Othercomputers/내 노트북/인공지능/Kaggle 참여/2. Binary Prediction of Poisonous Mushrooms/playground-series-s4e8/train.csv
# 경로: /content/drive/Othercomputers/내 노트북/인공지능/Kaggle 참여/2. Binary Prediction of Poisonous Mushrooms/playground-series-s4e8/test.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Train = pd.read_csv("/content/drive/Othercomputers/내 노트북/인공지능/Kaggle 참여/2. Binary Prediction of Poisonous Mushrooms/playground-series-s4e8/train.csv")
Test = pd.read_csv("/content/drive/Othercomputers/내 노트북/인공지능/Kaggle 참여/2. Binary Prediction of Poisonous Mushrooms/playground-series-s4e8/test.csv")

train_df = pd.DataFrame(Train)
test_df = pd.DataFrame(Test)

In [3]:
IDtest = test_df["id"]
train_df.drop(labels=["id"], axis=1, inplace=True)
test_df.drop(labels=["id"], axis=1, inplace=True)
display(train_df.head(3))
display(test_df.head(3))

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,e,8.80,f,s,u,f,a,c,w,4.51,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w


,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a
1,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,2.00,b,g,n,f,NaN,c,n,6.18,3.14,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s


* 최빈값들과 그 비율이 train과 test셋에서 비슷함
* 만들어진 느낌이 나긴함

# 2. Outlier Detection

In [4]:
# Tukey method 이용해서 연속값들로부터 outlier 분석하기
# Age, SibSp, Parch, Fare
from collections import Counter

def detect_outlier(df, n: int, features: list):
    result_list = []

    for col in features:
        p25 = np.percentile(df[col], 25)
        p75 = np.percentile(df[col], 75)
        # Interquartile range(IQR)
        IQR = p75 - p25

        # outlier step
        outlier_step = 1.5 * IQR

        outlier_list = df[(df[col] < p25 - outlier_step) | (df[col] > p75 + outlier_step)].index

        result_list.extend(outlier_list) # append하면 오류임

    result_list = Counter(result_list).items()
    result = [k for k, v in result_list if v > n]
    return result

outlier = detect_outlier(train_df, 2, ["cap-diameter","stem-height","stem-width"])
print(outlier)

[]


# 3. Null value handling

In [5]:
# def drop_high_null(df):
#     null_rate = df.isnull().mean() * 100
#     high_null_cols = null_rate[null_rate >= 85].index

#     df = df.drop(labels=high_null_cols, axis=1)
#     return df

# train_df = drop_high_null(train_df)
# test_df = drop_high_null(test_df)

# print(train_df.shape[1])
# print(test_df.shape[1])

# 4. Duplicated/Weird value handling

In [6]:
# 중복이 문제되는 경우는 train뿐임
# test는 처리 안해야됨
# new_index 부여하자
train_shape = train_df.shape

train_df = train_df.drop_duplicates(ignore_index=True)

print("Drop 이후 Train 데이터의 중복 건수:", train_df.duplicated().sum())
print()
print("Train 데이터의 줄어든 데이터 건수:", train_shape[0] - train_df.shape[0])

Drop 이후 Train 데이터의 중복 건수: 0

Train 데이터의 줄어든 데이터 건수: 0


In [7]:
def replace_weird_to_nan(df, object_cols):
    alphabet_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    for col in object_cols:
        df[col] = df[col].apply(lambda x: x if type(x)==str and len(x)==1 and x in alphabet_list else 'error')

    return df

In [8]:
train_df   = replace_weird_to_nan(train_df, train_df.describe(include='object').columns)
test_df = replace_weird_to_nan(test_df, test_df.describe(include='object').columns)

# 5. Null Filling/Encoding

In [9]:
# train, test의 feature가 동일한지 확인
train_cols = train_df.columns.tolist()
train_cols.pop(0)
test_cols = test_df.columns.tolist()
print(len(train_cols) == len(test_cols))
print(train_cols == test_cols)

True
True


In [10]:
# target feature인 class는 Labe Encoding 해줄꺼기 때문에 빼
# 마지막에 inverse_transform 사용하기 위함
num_cols = train_df.select_dtypes('number').columns.tolist()
cat_cols = train_df.select_dtypes('object').columns.tolist()
cat_cols.pop(0)

print(num_cols)
print(cat_cols)

['cap-diameter', 'stem-height', 'stem-width']
['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']


In [11]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import matthews_corrcoef

In [12]:
for col in num_cols:
    train_df[col] = train_df[col].apply(lambda x: np.log(x) if x>0 else 0)
    test_df[col] = test_df[col].apply(lambda x: np.log(x) if x>0 else 0)

In [13]:
num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=3)),
    ('scaler', StandardScaler()),
    ('convert_to_float32', FunctionTransformer(lambda x: x.astype(np.float32)))
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

def fillingNull_encoding(df, preprocessor, training=True):
    if training:
        df_transformed = pd.DataFrame(preprocessor.fit_transform(df[num_cols + cat_cols]), columns=num_cols+cat_cols)
        df_transformed['class'] = df['class']
    else:
        df_transformed = pd.DataFrame(preprocessor.transform(df[num_cols + cat_cols]), columns=num_cols+cat_cols)

    display(df_transformed.isnull().sum())
    return df_transformed

In [14]:
changed_train_df = fillingNull_encoding(train_df, preprocessor, training=True)

,0
cap-diameter,0
stem-height,0
stem-width,0
cap-shape,0
cap-surface,0
cap-color,0
does-bruise-or-bleed,0
gill-attachment,0
gill-spacing,0
gill-color,0


In [15]:
changed_test_df = fillingNull_encoding(test_df, preprocessor, training=False)

,0
cap-diameter,0
stem-height,0
stem-width,0
cap-shape,0
cap-surface,0
cap-color,0
does-bruise-or-bleed,0
gill-attachment,0
gill-spacing,0
gill-color,0


# 변환 test data 저장

In [16]:
changed_test_df.to_csv('scaled_test_data.csv')

In [17]:
changed_train_df.head()

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,class
0,0.788156,-0.673298,0.735480,6.0,17.0,19.0,6.0,0.0,2.0,20.0,...,5.0,20.0,5.0,5.0,5.0,6.0,5.0,3.0,0.0,e
1,-0.152106,-0.519709,-0.281936,21.0,8.0,14.0,6.0,0.0,2.0,13.0,...,22.0,14.0,5.0,5.0,17.0,23.0,5.0,3.0,3.0,p
2,0.454154,0.392438,0.220116,6.0,17.0,1.0,6.0,21.0,2.0,20.0,...,17.0,13.0,5.0,5.0,5.0,6.0,5.0,11.0,3.0,e
3,-0.363748,-0.879285,-0.272895,6.0,22.0,7.0,6.0,17.0,5.0,7.0,...,5.0,20.0,5.0,5.0,5.0,6.0,5.0,3.0,2.0,e
4,0.213819,-1.416301,0.017688,21.0,11.0,20.0,6.0,3.0,5.0,20.0,...,5.0,20.0,5.0,5.0,5.0,6.0,5.0,7.0,0.0,e


In [18]:
# Target 값 LabelEncoding하기
le = LabelEncoder()
changed_train_df['class'] = le.fit_transform(changed_train_df['class'])
changed_train_df.head()

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,class
0,0.788156,-0.673298,0.735480,6.0,17.0,19.0,6.0,0.0,2.0,20.0,...,5.0,20.0,5.0,5.0,5.0,6.0,5.0,3.0,0.0,0
1,-0.152106,-0.519709,-0.281936,21.0,8.0,14.0,6.0,0.0,2.0,13.0,...,22.0,14.0,5.0,5.0,17.0,23.0,5.0,3.0,3.0,1
2,0.454154,0.392438,0.220116,6.0,17.0,1.0,6.0,21.0,2.0,20.0,...,17.0,13.0,5.0,5.0,5.0,6.0,5.0,11.0,3.0,0
3,-0.363748,-0.879285,-0.272895,6.0,22.0,7.0,6.0,17.0,5.0,7.0,...,5.0,20.0,5.0,5.0,5.0,6.0,5.0,3.0,2.0,0
4,0.213819,-1.416301,0.017688,21.0,11.0,20.0,6.0,3.0,5.0,20.0,...,5.0,20.0,5.0,5.0,5.0,6.0,5.0,7.0,0.0,0


In [19]:
# train, test dataset num_cols에서의 skew 값
for col in num_cols:
    print("Train {}의 skew: {:.4f}, 평균: {:.4f}, 분산: {:.4f}".format(col, changed_train_df[col].skew(), changed_train_df[col].mean(), changed_train_df[col].var()))
print("====================")
for col in num_cols:
    print("Test {}의 skew: {}, 평균: {:.4f}, 분산: {:.4f}".format(col, changed_test_df[col].skew(), changed_test_df[col].mean(), changed_test_df[col].var()))

Train cap-diameter의 skew: -0.5299, 평균: 0.0000, 분산: 1.0000
Train stem-height의 skew: 0.0046, 평균: 0.0000, 분산: 1.0000
Train stem-width의 skew: -0.5689, 평균: 0.0000, 분산: 1.0000
Test cap-diameter의 skew: -0.5265808919253121, 평균: -0.0015, 분산: 1.0011
Test stem-height의 skew: 0.008050287287527878, 평균: -0.0006, 분산: 0.9989
Test stem-width의 skew: -0.568084756451348, 평균: -0.0008, 분산: 1.0002


# 6. Modeling
* 실험 편리를 위해 changed_train_df, chagned_test_df 사용

In [20]:
pip install xgboost=='1.7.3'

In [21]:
y_data = changed_train_df['class']
x_data = changed_train_df.drop(labels=['class'], axis=1, inplace=False)

In [22]:
def mcc_metric(y_pred, dmatrix):
    y_true = dmatrix.get_label()
    y_pred = (y_pred > 0.5).astype(int)
    mcc = matthews_corrcoef(y_true, y_pred)
    return 'mcc', mcc

In [23]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.base import clone

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFECV

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, confusion_matrix, matthews_corrcoef, make_scorer

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [24]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [25]:
import pickle

def cross_validate_score(model, model_name, train_df, y, cv, test_data):
    val_scores = []
    test_preds = np.zeros((test_data.shape[0],))
    oof_preds = np.zeros((train_df.shape[0],))

    for fold, (train_idx, val_idx) in enumerate(cv.split(train_df, y)):
        X_train = train_df.iloc[train_idx].reset_index(drop=True)
        y_train = y.iloc[train_idx].reset_index(drop=True)

        X_val = train_df.iloc[val_idx].reset_index(drop=True)
        y_val = y.iloc[val_idx].reset_index(drop=True)

        model = clone(model)

        eval_set = [(X_val, y_val)]

        if isinstance(model, LGBMClassifier):
            model.fit(
                X_train, y_train,
                eval_set=eval_set
            )
        elif isinstance(model, XGBClassifier):
            model.fit(
                X_train, y_train,
                eval_set=eval_set,
                eval_metric = mcc_metric,
                verbose=True
            )
        else:
            model.fit(
                X_train, y_train,
            )

        with open('{}_{}fold.pickle'.format(model_name, fold), 'wb') as fw:
            pickle.dump(model, fw)

        val_probs = model.predict_proba(X_val)[:, 1]  # Get the probabilities
        val_preds = (val_probs > 0.5).astype(int)     # Convert probabilities to class labels for MCC

        val_score = matthews_corrcoef(y_val, val_preds)  # Calculate MCC
        print(f'Fold {fold}: MCC = {val_score:.5f}')

        val_scores.append(val_score)

        oof_preds[val_idx] = val_probs  # Store the probabilities for OOF predictions

        test_preds += model.predict_proba(test_data)[:, 1] / cv.get_n_splits()  # Aggregate test probabilities


    mean_val_score = np.mean(val_scores)
    std_val_score = np.std(val_scores)
    print(f'Mean Validation MCC: {mean_val_score:.7f}')
    print(f'Std Validation MCC: {std_val_score:.7f}')

    return val_scores, test_preds, oof_preds

In [26]:
cv_summary, test_preds, oof_preds = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

# XGBoost

In [27]:
xgb_params = {
    'tree_method': 'gpu_hist',
 'n_estimators': 1000,
 'subsample': 0.8,
 'colsample_bytree': 0.4,
 'max_depth': 15,
 'min_child_weight': 5,
 'learning_rate': 0.02
}

xgb_tuned = XGBClassifier(**xgb_params, random_state=42)

cv_summary['xgb'], test_preds['xgb'], oof_preds['xgb'] = cross_validate_score(xgb_tuned, 'XGB', x_data , y_data,  cv, changed_test_df)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.67807	validation_0-mcc:0.84286
[1]	validation_0-logloss:0.66208	validation_0-mcc:0.94694
[2]	validation_0-logloss:0.64848	validation_0-mcc:0.95729
[3]	validation_0-logloss:0.63218	validation_0-mcc:0.97142
[4]	validation_0-logloss:0.61695	validation_0-mcc:0.97332
[5]	validation_0-logloss:0.60544	validation_0-mcc:0.97303
[6]	validation_0-logloss:0.59289	validation_0-mcc:0.97441
[7]	validation_0-logloss:0.58162	validation_0-mcc:0.97431
[8]	validation_0-logloss:0.56789	validation_0-mcc:0.97835
[9]	validation_0-logloss:0.55879	validation_0-mcc:0.97738
[10]	validation_0-logloss:0.54762	validation_0-mcc:0.97856
[11]	validation_0-logloss:0.53511	validation_0-mcc:0.97908
[12]	validation_0-logloss:0.52460	validation_0-mcc:0.97966
[13]	validation_0-logloss:0.51467	validation_0-mcc:0.97926
[14]	validation_0-logloss:0.50359	validation_0-mcc:0.97921
[15]	validation_0-logloss:0.49254	validation_0-mcc:0.98056
[16]	validation_0-logloss:0.48241	validation_0-mcc:0.98102
[17]	va

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.67805	validation_0-mcc:0.84310
[1]	validation_0-logloss:0.66204	validation_0-mcc:0.94667
[2]	validation_0-logloss:0.64847	validation_0-mcc:0.95617
[3]	validation_0-logloss:0.63218	validation_0-mcc:0.97055
[4]	validation_0-logloss:0.61660	validation_0-mcc:0.97516
[5]	validation_0-logloss:0.60517	validation_0-mcc:0.97478
[6]	validation_0-logloss:0.59260	validation_0-mcc:0.97655
[7]	validation_0-logloss:0.58133	validation_0-mcc:0.97695
[8]	validation_0-logloss:0.56762	validation_0-mcc:0.97872
[9]	validation_0-logloss:0.55849	validation_0-mcc:0.97852
[10]	validation_0-logloss:0.54731	validation_0-mcc:0.97951
[11]	validation_0-logloss:0.53483	validation_0-mcc:0.97984
[12]	validation_0-logloss:0.52400	validation_0-mcc:0.98040
[13]	validation_0-logloss:0.51412	validation_0-mcc:0.98007
[14]	validation_0-logloss:0.50305	validation_0-mcc:0.98010
[15]	validation_0-logloss:0.49202	validation_0-mcc:0.98075
[16]	validation_0-logloss:0.48192	validation_0-mcc:0.98163
[17]	va

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.67806	validation_0-mcc:0.84328
[1]	validation_0-logloss:0.66206	validation_0-mcc:0.94642
[2]	validation_0-logloss:0.64848	validation_0-mcc:0.95647
[3]	validation_0-logloss:0.63219	validation_0-mcc:0.97048
[4]	validation_0-logloss:0.61669	validation_0-mcc:0.97540
[5]	validation_0-logloss:0.60516	validation_0-mcc:0.97467
[6]	validation_0-logloss:0.59261	validation_0-mcc:0.97652
[7]	validation_0-logloss:0.58133	validation_0-mcc:0.97689
[8]	validation_0-logloss:0.56762	validation_0-mcc:0.97867
[9]	validation_0-logloss:0.55851	validation_0-mcc:0.97846
[10]	validation_0-logloss:0.54734	validation_0-mcc:0.97920
[11]	validation_0-logloss:0.53483	validation_0-mcc:0.97979
[12]	validation_0-logloss:0.52411	validation_0-mcc:0.98042
[13]	validation_0-logloss:0.51419	validation_0-mcc:0.97994
[14]	validation_0-logloss:0.50307	validation_0-mcc:0.98022
[15]	validation_0-logloss:0.49204	validation_0-mcc:0.98087
[16]	validation_0-logloss:0.48195	validation_0-mcc:0.98159
[17]	va

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.67808	validation_0-mcc:0.84053
[1]	validation_0-logloss:0.66208	validation_0-mcc:0.94660
[2]	validation_0-logloss:0.64850	validation_0-mcc:0.95615
[3]	validation_0-logloss:0.63222	validation_0-mcc:0.97120
[4]	validation_0-logloss:0.61671	validation_0-mcc:0.97494
[5]	validation_0-logloss:0.60497	validation_0-mcc:0.97470
[6]	validation_0-logloss:0.59242	validation_0-mcc:0.97642
[7]	validation_0-logloss:0.58117	validation_0-mcc:0.97679
[8]	validation_0-logloss:0.56745	validation_0-mcc:0.97854
[9]	validation_0-logloss:0.55838	validation_0-mcc:0.97852
[10]	validation_0-logloss:0.54726	validation_0-mcc:0.97931
[11]	validation_0-logloss:0.53473	validation_0-mcc:0.97976
[12]	validation_0-logloss:0.52391	validation_0-mcc:0.98033
[13]	validation_0-logloss:0.51403	validation_0-mcc:0.97978
[14]	validation_0-logloss:0.50291	validation_0-mcc:0.97981
[15]	validation_0-logloss:0.49191	validation_0-mcc:0.98059
[16]	validation_0-logloss:0.48181	validation_0-mcc:0.98112
[17]	va

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.67806	validation_0-mcc:0.84217
[1]	validation_0-logloss:0.66204	validation_0-mcc:0.94710
[2]	validation_0-logloss:0.64845	validation_0-mcc:0.95686
[3]	validation_0-logloss:0.63216	validation_0-mcc:0.97055
[4]	validation_0-logloss:0.61666	validation_0-mcc:0.97535
[5]	validation_0-logloss:0.60517	validation_0-mcc:0.97503
[6]	validation_0-logloss:0.59262	validation_0-mcc:0.97678
[7]	validation_0-logloss:0.58136	validation_0-mcc:0.97701
[8]	validation_0-logloss:0.56764	validation_0-mcc:0.97907
[9]	validation_0-logloss:0.55855	validation_0-mcc:0.97885
[10]	validation_0-logloss:0.54739	validation_0-mcc:0.97994
[11]	validation_0-logloss:0.53488	validation_0-mcc:0.98016
[12]	validation_0-logloss:0.52428	validation_0-mcc:0.98055
[13]	validation_0-logloss:0.51436	validation_0-mcc:0.98003
[14]	validation_0-logloss:0.50326	validation_0-mcc:0.98007
[15]	validation_0-logloss:0.49223	validation_0-mcc:0.98088
[16]	validation_0-logloss:0.48213	validation_0-mcc:0.98156
[17]	va

# LightGBM

In [28]:
lgbm_params = {
    'n_estimators': 2500,
    'max_bin':1024,
    'colsample_bytree':0.6,
    'reg_lambda': 80,
    'verbosity': -1
}

lgbm_tuned = LGBMClassifier(**lgbm_params, random_state=42, verbose=-1)

cv_summary['lgbm'], test_preds['lgbm'], oof_preds['lgbm'] = cross_validate_score(lgbm_tuned, 'LGBM', x_data , y_data,  cv, changed_test_df)

Fold 0: MCC = 0.98439
Fold 1: MCC = 0.98421
Fold 2: MCC = 0.98433
Fold 3: MCC = 0.98423
Fold 4: MCC = 0.98445
Mean Validation MCC: 0.9843227
Std Validation MCC: 0.0000924


# Catboost

In [ ]:
# catb_params = {
#     "n_estimators" : 2500,
#     "learning_rate" : 0.13,
#     'max_bin': 1024,
#     'grow_policy': 'SymmetricTree',
#     'boosting_type': 'Plain',
#     'bootstrap_type': 'Bernoulli',
# }

# catb_tunned = CatBoostClassifier(**catb_params, random_state=42)

# cv_summary['catb'], test_preds['catb'], oof_preds['catb'] = cross_validate_score(catb_tunned, x_data , y_data,  cv, changed_test_df)

In [29]:
cv_summary, test_preds, oof_preds

(        xgb      lgbm
 0  0.984791  0.984390
 1  0.984834  0.984213
 2  0.984853  0.984326
 3  0.984680  0.984230
 4  0.984855  0.984455,
               xgb      lgbm
 0        0.002253  0.002030
 1        0.998073  0.999005
 2        0.994812  0.992582
 3        0.996157  0.995519
 4        0.001078  0.001013
 ...           ...       ...
 2077959  0.997944  0.997604
 2077960  0.991851  0.994287
 2077961  0.993814  0.993896
 2077962  0.001806  0.001600
 2077963  0.001283  0.002521
 
 [2077964 rows x 2 columns],
               xgb      lgbm
 0        0.001860  0.002063
 1        0.997494  0.997547
 2        0.000599  0.000533
 3        0.006865  0.008754
 4        0.014900  0.011495
 ...           ...       ...
 3116940  0.001296  0.001292
 3116941  0.011756  0.006916
 3116942  0.996670  0.992368
 3116943  0.009919  0.006028
 3116944  0.996988  0.997438
 
 [3116945 rows x 2 columns])

# Stacking

In [30]:
meta_model_params = {
    'C': 0.000237302749626327,
    'max_iter': 2500,
    'tol': 9.996751434702547e-05,
    'solver': 'saga',
    'penalty': 'l1'
}

meta_model = LogisticRegression(**meta_model_params, random_state=42)

In [31]:
min_features_to_select = 1

# Create a pipeline with preprocessor and RFECV
pipeline = Pipeline([

    ('Scaler', StandardScaler()),
    ('rfecv', RFECV(estimator=meta_model,
                    step=1,
                    cv=cv,
                    scoring=make_scorer(matthews_corrcoef),
                    min_features_to_select=min_features_to_select,
                    n_jobs=-1,))
])

# Fit the pipeline on oof_preds
pipeline.fit(oof_preds, y_data)

#CV score
print("Best CV score: ")
selected_models = np.array( oof_preds.columns)[pipeline.named_steps['rfecv'].support_]
print( pipeline.named_steps['rfecv'].cv_results_["mean_test_score"][len(selected_models) - 1])


# Selected models after RFECV
print('Number of available models:', len(oof_preds.columns))
print('Number of selected models for ensemble:', len(selected_models))
print("Selected models:", selected_models)

Best CV score: 
0.9848126458591718
Number of available models: 2
Number of selected models for ensemble: 1
Selected models: ['xgb']


In [32]:
meta_model = meta_model.fit(oof_preds[selected_models], y_data)

# 제출 파일 만들기

In [ ]:
pred_class = le.inverse_transform(preds_test)
pred_class

In [ ]:
submission = pd.DataFrame(IDtest, columns=['id'])
submission['class'] = pred_class

In [ ]:
submission.to_csv('sub_stacking.csv', index=False)
pd.read_csv('sub_stacking.csv')

In [ ]:
"""
첫 실험,
xgb = XGBClassifier(
    n_estimators=400,
    max_depth=15,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.4,
    min_child_weight=5,
)

metric = mcc

validation_0-logloss:0.06993	validation_0-mcc:0.98150
public score -> 0.98194
"""

"""
두번째 실험,
동일 모델,
eval_metric = 'logloss' 만 다름

결과:
validation_0-logloss:0.06993
public score -> 0.98194 동일...
"""

"""
세번째 실험,
동일 모델,
metric = mcc

결과:
validation_0-logloss:0.06791	validation_0-mcc:0.98184
public score -> 0.98204
"""

"""
네번째 실험,
n_estimators=1000
동일 모델,
metric = mcc

결과:
validation_0-logloss:0.03977	validation_0-mcc:0.98286
public score -> 0.98312
"""

"""
다섯번째 실험,
weird 값을 "error"라는 새로운 값으로 처리
learning_rate = 0.01 그대로 유지

결과:
validation_0-logloss:0.03733	validation_0-mcc:0.98425
public score -> 0.98464
"""


"""
여섯번째 실험,
weird 값을 "error"라는 새로운 값으로 처리
learning_rate = 0.02

결과:
validation_0-logloss:0.03708	validation_0-mcc:0.98419
public score -> 0.98478
"""

"""
일곱번째 실험,
Straight KFold, n_splits=5로 xgb 실험

결과:
Mean Validation MCC: 0.9844321
public score -> 0.98472

"""


"""
여덟번째 실험, 파라미터 안좋네 다시 복
Straight KFold, n_splits=5로 xgb2 실험

결과:
Mean Validation MCC: 0.9842396
public score -> 0.98453
"""